In [1]:
import openai
import json
import numpy as np
from numpy.linalg import norm
import requests
import csv
import tiktoken
from pprint import pprint
from collections import defaultdict
api_key = open("api_key").read()
openai.api_key = api_key
def get_embedding(text, model="text-embedding-ada-002"):
   encoder = tiktoken.encoding_for_model(model)
   token_length = len(encoder.encode(text))
   while token_length > 8191:
      text = text[:-100]
      token_length = len(encoder.encode(text))
      print(token_length)
   # text = text.replace("\n", " ")
   # return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
   url = 'https://api.openai.com/v1/embeddings'
   headers = {
      'Content-Type': 'application/json',
      'Authorization': "Bearer {}".format(api_key)
   }
   data = {
      "input": text,
      "model": model
   }
   res = requests.post(url, headers=headers, json=data)
   res = res.json()
   return res['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [ ]:
import json
from collections import defaultdict
pairwise_evaluation = json.load(open('pairwise_evaluation_w_embeddings.json'))
pairwise_evaluation_dict = defaultdict(list)
for item in pairwise_evaluation:
    pairwise_evaluation_dict[item['article_id']].append(item)

In [ ]:
from pprint import pprint
writer_samples = {}
for article_id, data in pairwise_evaluation_dict.items():
    writer_samples[article_id] = data[0]
save_json(writer_samples, 'pairwise_samples.json')


In [1]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from scipy import spatial
import math 
from sklearn.cluster import OPTICS
def compute_transformation(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    # A = X / Y # Does not work
    # A = np.tanh(X) / Y # Does not work
    # X = np.array([1/(1+math.exp(-x)) for x in X]) # Does not work
    # X = np.array([math.exp(-x**2) for x in X]) # Does not work
    
    # A = X / Y 
    A = X - Y
    # print((np.min(X), np.max(X)), (np.min(Y), np.max(Y)))
    
    assert(X.shape == A.shape)
    return A

def _compute_transformation(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    A = np.concatenate((X, Y))
    print(A.shape)
    return A

def k_means(X, k):
    X = np.array(X)
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)

    # Get the cluster centers (representative matrices)
    cluster_centers = kmeans.cluster_centers_

    # Assign each matrix to its nearest cluster
    labels, _ = pairwise_distances_argmin_min(X, cluster_centers)

    # Print the cluster assignments for each matrix
    # for i, label in enumerate(labels):
    #     print(f"Matrix {i + 1} is assigned to Cluster {label + 1}")
    return labels

def optics(X):
    clustering = OPTICS(min_samples=2).fit(X)
    return clustering.labels_, clustering.cluster_hierarchy_

def cos_distance_matrix(X):
    D = np.zeros((X.shape[0], X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(X.shape[0]):
            print(i, j, X.shape[0])
            D[i][j] = spatial.distance.cosine(X[i], X[j])
    return D

def labels_to_clusters(labels):
    clusters = defaultdict(list)
    for index, label in enumerate(labels):
        clusters[label].append(index)
    return clusters

In [5]:
pairwise_evaluation = json.load(open('pairwise_samples.json'))
better_summary_embeddings = [sample['writer_summary_embedding'] if sample['overall_writer_better'] else sample['llm_summary_embedding'] for sample in pairwise_evaluation.values() ]
worse_summary_embeddings = [sample['writer_summary_embedding'] if not sample['overall_writer_better'] else sample['llm_summary_embedding'] for sample in pairwise_evaluation.values() ]
better_transformations = []
worse_transformations = []
# for sample in pairwise_evaluation.values():
    # A1, residuals = estimate_transformation(sample['full_embedding'], sample['llm_summary_embedding'])
    # transformations.append(A1)
    # A2 = _compute_transformation(sample['full_embedding'], sample['writer_summary_embedding'])
    # transformations.append(A2)
# print(len(transformations))

In [ ]:
pairwise_evaluation = json.load(open('pairwise_samples.json'))
better_summary_embeddings = [(sample['full_embedding'], sample['writer_summary_embedding']) if sample['overall_writer_better'] else (sample['full_embedding'], sample['llm_summary_embedding']) for sample in pairwise_evaluation.values() ]
worse_summary_embeddings = [(sample['full_embedding'], sample['writer_summary_embedding']) if not sample['overall_writer_better'] else (sample['full_embedding'], sample['llm_summary_embedding']) for sample in pairwise_evaluation.values() ]
better_transformations = [compute_transformation(full_embedding, summary_embedding) for full_embedding, summary_embedding in better_summary_embeddings]
worse_transformations = [compute_transformation(full_embedding, summary_embedding) for full_embedding, summary_embedding in worse_summary_embeddings]
better_transformation_kmeans = k_means(np.array(better_transformations), 5)
worse_transformation_kmeans = k_means(np.array(worse_transformations), 5)
better_transformation_clusters = labels_to_clusters(better_transformation_kmeans)
worse_transformation_clusters = labels_to_clusters(worse_transformation_kmeans)

In [ ]:
transformation_kmeans = k_means(np.array(transformations), 5)
print("transformation kmeans done")
full_kmeans = k_means(np.array([sample['full_embedding'] for sample in pairwise_evaluation.values()]), 5)
print("full kmeans done")
writer_summary_kmeans = k_means(np.array([sample['writer_summary_embedding'] for sample in pairwise_evaluation.values()]), 5)
print("writer summary kmeans done")

In [ ]:
transformation_clusters = labels_to_clusters(transformation_kmeans)
full_clusters = labels_to_clusters(full_kmeans)
writer_summary_clusters = labels_to_clusters(writer_summary_kmeans)

In [ ]:
print(list(pairwise_evaluation.values())[4]['writer_summary'])
print(list(pairwise_evaluation.values())[32]['writer_summary'])
print(list(pairwise_evaluation.values())[35]['writer_summary'])
print(list(pairwise_evaluation.values())[46]['writer_summary'])
print(list(pairwise_evaluation.values())[54]['writer_summary'])
print(list(pairwise_evaluation.values())[59]['writer_summary'])
print(list(pairwise_evaluation.values())[65]['writer_summary'])

In [ ]:
print(list(pairwise_evaluation.values())[16]['writer_summary'])
print(list(pairwise_evaluation.values())[28]['writer_summary'])
print(list(pairwise_evaluation.values())[29]['writer_summary'])
print(list(pairwise_evaluation.values())[43]['writer_summary'])
print(list(pairwise_evaluation.values())[45]['writer_summary'])
print(list(pairwise_evaluation.values())[56]['writer_summary'])
print("==================")
print(list(pairwise_evaluation.values())[2]['writer_summary'])
print(list(pairwise_evaluation.values())[12]['writer_summary'])
print(list(pairwise_evaluation.values())[13]['writer_summary'])
print(list(pairwise_evaluation.values())[20]['writer_summary'])

In [12]:
for cluster_indicies in better_summary_clusters.values():
    summaries = [list(pairwise_evaluation.values())[index]['writer_summary'] for index in cluster_indicies]
    print(len(summaries))
    print("==================")

20
25
14
6
11
